In [11]:
import zipfile
import os
import wfdb
import numpy as np
from scipy.signal import resample
import biosppy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from concurrent.futures import ThreadPoolExecutor

def extract_zip(zip_path, extract_to):
    """Распаковывает ZIP-архив, если папка ещё не создана."""
    if not os.path.exists(extract_to):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)

def load_ecg_file(file_path):
    """Загружает одиночный файл ЭКГ."""
    try:
        record = wfdb.rdrecord(file_path)
        return record.p_signal.flatten()
    except Exception as e:
        print(f"Ошибка загрузки {file_path}: {e}")
        return None

def load_ecg_data_from_folder(folder_path, limit=100, max_workers=4):
    """Загружает ограниченное количество ЭКГ файлов с ограничением потоков."""
    ecg_files = []
    for root, _, files in os.walk(folder_path):
        dat_files = [os.path.join(root, f.split('.')[0]) for f in files if f.endswith('.dat')]
        ecg_files.extend(dat_files)
    
    ecg_files = ecg_files[:limit]  # Ограничиваем количество загружаемых файлов
    
    ecg_data = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for data in executor.map(load_ecg_file, ecg_files):
            if data is not None:
                ecg_data.append(data)
    
    return ecg_data


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\Lenovo\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Lenovo\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\Lenovo\anaconda3\lib\site

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
def process_ecg_signal(signal, target_fs=1000, max_length=600):
    """Обрабатывает одиночный ЭКГ-сигнал, ограничивая длину."""
    if len(signal) == 0:
        return np.array([])

    # Укорачиваем сигнал до max_length (например, 600 точек)
    if len(signal) > max_length:
        signal = signal[:max_length]

    # Интерполяция
    resampled = resample(signal, int(len(signal) * target_fs / 1000))

    # Простая фильтрация и расчет признаков
    filtered, _, _ = biosppy.tools.filter_signal(signal=resampled, ftype='butter', band='lowpass', order=2, frequency=40, sampling_rate=target_fs)
    heart_rate = np.mean(np.abs(np.diff(filtered)))  

    return np.array([heart_rate])

def process_ecg_data(ecg_data, target_fs=1000):
    """Обрабатывает список ЭКГ-сигналов в многопоточном режиме."""
    with ThreadPoolExecutor() as executor:
        features = list(executor.map(lambda s: process_ecg_signal(s, target_fs), ecg_data))
    
    return np.array([f for f in features if f.size > 0])  # Убираем пустые массивы

def train_model(X, y):
    """Обучает модель логистической регрессии."""
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')
    return clf

def predict_ecg(model, ecg_signal):
    """Делает предсказание для нового сигнала ЭКГ."""
    features = process_ecg_signal(ecg_signal)
    if features.size == 0:
        return "Ошибка обработки"
    prediction = model.predict([features])
    return "Больной" if prediction[0] == 1 else "Здоров"

In [ ]:
# Пути к данным
atrial_fibrillation_zip = 'mit-bih-atrial-fibrillation-database-1.0.0.zip'
normal_sinus_rhythm_zip = 'mit-bih-normal-sinus-rhythm-database-1.0.0.zip'

a_fib_dir = 'atrial_fibrillation_data'
norm_sinus_dir = 'normal_sinus_rhythm_data'

# Распаковка (ускоренная)
extract_zip(atrial_fibrillation_zip, a_fib_dir)
extract_zip(normal_sinus_rhythm_zip, norm_sinus_dir)

atrial_fibrillation_data = load_limited_ecg_data_from_folder(a_fib_dir, limit=200)
normal_sinus_rhythm_data = load_limited_ecg_data_from_folder(norm_sinus_dir, limit=200)

# Ограничиваем обработку (не более 50 сигналов за раз)
batch_size = 50
atrial_fibrillation_features = []
normal_sinus_rhythm_features = []

for i in range(0, len(atrial_fibrillation_data), batch_size):
    batch = atrial_fibrillation_data[i:i+batch_size]
    atrial_fibrillation_features.extend(process_ecg_data(batch))

for i in range(0, len(normal_sinus_rhythm_data), batch_size):
    batch = normal_sinus_rhythm_data[i:i+batch_size]
    normal_sinus_rhythm_features.extend(process_ecg_data(batch))

# Метки
labels_atrial = np.ones(len(atrial_fibrillation_features))
labels_normal = np.zeros(len(normal_sinus_rhythm_features))

# Объединение данных
features = np.vstack((atrial_fibrillation_features, normal_sinus_rhythm_features))
labels = np.concatenate((labels_atrial, labels_normal))

# Обучение модели
model = train_model(features, labels)

# Тестовая предсказательная функция
sample_signal = atrial_fibrillation_data[0]  # Берем один сигнал для теста
print(f'Результат предсказания: {predict_ecg(model, sample_signal)}')